In [ ]:
#import all the package needed
#original
#use augmention
#use dropout
#remove the dropout layer before the final layer
#add more FC layers for hair color recognition
# torch.nn.ConvTranspose2d can do unsampling and can be regarded as a deconvolution operation.
from sklearn.manifold import TSNE
from __future__ import print_function
import torch
import numpy as np
import MNISTtools as mit
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.autograd import Variable
from __future__ import print_function
import torch.utils.data as Data
import torchvision
import os
import matplotlib.pylab as plt
# import Augmentor

In [ ]:
class Net_1(nn.Module):
    def __init__(self):
        super(Net_1,self).__init__()
        self.conv1 = nn.Conv2d(3,6,3)
        torch.nn.init.xavier_uniform(self.conv1.weight)
        self.conv1_bn = nn.BatchNorm2d(6)
        
        self.conv2 = nn.Conv2d (6,16,3)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.conv2_bn = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(16,30,3)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.conv3_bn = nn.BatchNorm2d(30)   
        
        self.conv4 = nn.Conv2d(30,80,3)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.conv4_bn = nn.BatchNorm2d(80)
        
        self.conv5 = nn.Conv2d(80,100,3)
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.conv5_bn = nn.BatchNorm2d(100)
###fully connected layer 
        
        self.fc1 = nn.Linear(1200,1000)
        self.fc1_drop = nn.Dropout()
        torch.nn.init.xavier_uniform(self.fc1.weight)
        self.fc1_bn = nn.BatchNorm2d(1000)  
        self.fc2 = nn.Linear (1000,800)
        self.fc2_drop = nn.Dropout()
        torch.nn.init.xavier_uniform(self.fc2.weight)
        self.fc2_bn = nn.BatchNorm2d(800)    
        self.fc3 = nn.Linear (800,500)
        torch.nn.init.xavier_uniform(self.fc3.weight)
        self.fc3_bn = nn.BatchNorm2d(500)       
        self.fc4 = nn.Linear (500,4)
        torch.nn.init.xavier_uniform(self.fc3.weight)
        
###forward 
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1_bn(self.conv1(x))),(2,2))
        x = F.max_pool2d(F.relu(self.conv2_bn(self.conv2(x))),(2,2))
        x = F.max_pool2d(F.relu(self.conv3_bn(self.conv3(x))),(2,2))
        x = F.max_pool2d(F.relu(self.conv4_bn(self.conv4(x))),(2,2))
        x = F.max_pool2d(F.relu(self.conv5_bn(self.conv5(x))),(2,2))
        x = x.view(-1,self.num_flat_features(x))
        #print (x.size())
        x = self.fc1_drop(F.relu(self.fc1_bn(self.fc1(x))))
        x = self.fc2_drop(F.relu(self.fc2_bn(self.fc2(x))))
        x = F.relu(self.fc3_bn(self.fc3(x)))
        output = self.fc4(x)
        #output = x
        return output
    
    def num_flat_features(self,x):
        size = x.size()[1:]
        return np.prod(size)
    
def label2onehot(lbl):                     
    d = np.zeros((lbl.max() + 1, lbl.size)) 
    d[lbl, np.arange(0, lbl.size)] = 1 
    return d

#this function is written for assignment#2,so to be modified
def normalize_MNIST_images(x):
    num_f = x.shape[0]
    x=x.astype(np.float64)
    mean_each_feature = np.mean(x,axis = 1).reshape(num_f,1)
    result = (x - mean_each_feature)/255
    return result,mean_each_feature

def accuracy(prediction,lable):
    assert len(prediction) == len(lable)
    return np.sum(prediction == lable).astype(float)/len(prediction)
    
def onehot2label(d):
    lbl = d.argmax(axis=0) 
    return lbl    

def Get_Desktop_Path():
    return os.path.join(os.path.expanduser("~"), 'Desktop')

def load_images(index_to_use):
    im = np.zeros((218,178,3,len(index_to_use)))
    for i in range(202599):
        if i in index_to_use:
            index = i
            i = '0'*(6-len(str(i+1)))+str(i+1)
            im[:,:,:,index_to_use.index(index)] = plt.imread('/datasets/ee285s-public/CelebA/Img/img_align_celeba/%s.jpg'%i)
    return im

#do the selection and preprocessing for labels,also could get which images(images' index) within dataset is qualified
def load_labels():
    # Bald             4
    # Black_Hair       8 
    # Blond_Hair       9      
    # Brown_Hair       11
    # Gray_Hair       17
    # Straight_Hair   32
    # Wavy_Hair       33
    # Wearing_Hat     35
    with open('/datasets/ee285s-public/CelebA/Anno/list_attr_celeba.txt','r')  as f:
        p_temp=f.readlines()
    p = []
    for i in p_temp:
        i = i[11:]
        i = i[:-2]
        i = i.replace(' ','')
        i = i.replace('-1','0')
        p.append(i)
    p = p[2:]   
    ylabel_temp = np.zeros((202599,40))
    index = 0
    for each in p:
        temp_x = []
        for i in each:
            temp_x.append(int(i))
        ylabel_temp[index,:] = np.array(temp_x)
        index += 1
    ylabel_for_color = ylabel_temp[:,[8,9,11,17]]
    ylabel_for_style = ylabel_temp[:,[4,32,33,35]]
    index_to_delete = image_to_abandon(ylabel_for_color,ylabel_for_style)
    index_to_use = np.delete(np.arange(202599),index_to_delete)
    return ylabel_for_color,ylabel_for_style,index_to_use.tolist()

def image_to_abandon(ylabel_for_color,ylabel_for_style):
    which_images_to_delete_1_1 = np.where(np.sum(ylabel_for_color,1) == 0)[0].tolist()
    which_images_to_delete_1_2 = np.where(np.sum(ylabel_for_style,1) == 0)[0].tolist()
    which_images_to_delete_2_1 = np.where(np.sum(ylabel_for_color,1) == 2)[0].tolist()
    which_images_to_delete_2_2 = np.where(np.sum(ylabel_for_color,1) == 3)[0].tolist()
    which_images_to_delete_2_3 = np.where(np.sum(ylabel_for_color,1) == 4)[0].tolist()
    which_images_to_delete_3_1 = np.where(np.sum(ylabel_for_style,1) == 2)[0].tolist()
    which_images_to_delete_3_2 = np.where(np.sum(ylabel_for_style,1) == 3)[0].tolist()
    which_images_to_delete_3_3 = np.where(np.sum(ylabel_for_style,1) == 4)[0].tolist()
    result = set(which_images_to_delete_1_1 + which_images_to_delete_1_2 + which_images_to_delete_2_1 + which_images_to_delete_2_2 + which_images_to_delete_2_3 + which_images_to_delete_3_1 + which_images_to_delete_3_2 + which_images_to_delete_3_3)
    return list(result)

#to randomly assign images' index into different batches per epoch
def get_the_step_indexassignment(train_family,batch):
    index_to_use = train_family
    np.random.shuffle(index_to_use)
    per_batch = family_of_index_to_use(index_to_use,batch)
    num_batch = len(per_batch)
    return per_batch,num_batch

#a subfunction within get_the_step_indexassignment,it's function is grouping the selected index
#into different batchs.
def family_of_index_to_use(index_to_use,batch = 1000):
    start = batch
    end = start + batch
    family_of_index_to_use = [index_to_use[0:batch]]
    while len(family_of_index_to_use[-1]) >= batch:
        family_of_index_to_use.append(index_to_use[start:end])
        start += batch
        end += batch
    family_of_index_to_use = family_of_index_to_use[:-1]
    return family_of_index_to_use
    
#use the assigned index to call the selected images from dataset.  
def get_x_for_thisbatch(perbatch_index,signal):
    if signal == False:
        x = load_images(perbatch_index)
        x = torch.from_numpy(np.moveaxis(np.moveaxis(x,-1,0),-1,1)).float()
    else:
        x = load_created_images(perbatch_index)
        x = torch.from_numpy(np.moveaxis(np.moveaxis(x,-1,0),-1,1)).float()
    return x

def load_created_images(perbatch_index):
    im = np.zeros((218,178,3,len(index_to_use)))
    for i in range(202599):
        if i in index_to_use:
            index = i
            i = '0'*(6-len(str(i+1)))+str(i+1)
            im[:,:,:,index_to_use.index(index)] = plt.imread('/datasets/ee285s-public/CelebA/Augmentation/%s.jpg'%i)
    return im

#call the labels    
def get_y_for_thisbatch(perbatch_index,ylabel):
    y = ylabel[perbatch_index,:].argmax(axis=1)
    y = torch.from_numpy(y).type(torch.LongTensor)  
    return y

def average(x):
    result = []
    for each in x:
        temp = each/2
        result.append(temp)
    return result

def do_some_statistics(prediction,label,chars):
    result = []
    for i in range(len(chars)):
        result.append(0)
    mis_pre = label[np.where((prediction != label) == 1)[0].tolist()]
    i = 0
    for each in chars:
        for pre in mis_pre:
            if pre == each:
                result[i] += 1
        i+=1
    return result
        
#this part of code is written to do the image augmentation locally
def augmentor():    
    p = Augmentor.Pipeline('/datasets/ee285s-public/CelebA/Img/img_align_celeba')
    p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=8)
    p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    p.flip_left_right(probability=1)
    p.zoom(probability=0.5, min_factor=1.2, max_factor=1.3)
    p.gaussian_distortion(probability = 1,grid_width = 5,grid_height = 5,magnitude = 5,corner = 'bell',method = 'in')
    p.skew(probability = 0.5)
    p.process()
    p.sample(20)
    
def batch_rename_for_augmentor():
    i=0
    path='/Users/sdret/Desktop/image/img_align_celeba/output'
    filelist=os.listdir(path)
    for files in filelist:
        i=i+1
        Olddir=os.path.join(path,files)
        filename=os.path.splitext(files)[0]
        filetype=os.path.splitext(files)[1]
        Newdir=os.path.join('/Users/sdret/Desktop/image/img_align_celeba/output',files[26:32]+filetype);
        os.rename(Olddir,Newdir)

def get_num_of_each_labels(index_family,label_type):
    if label_type == 'ylabel_for_color':
        temp = get_y_for_thisbatch(index_family,ylabel_for_color).numpy()
    else:
        temp = get_y_for_thisbatch(index_family,ylabel_for_style).numpy()
    return np.sum(label2onehot(temp),1)

def get_the_mis_ratio(result,col_or_sty):
    if col_or_sty == 'col':
        num_test_col = np.array([4993,3776,5048,700])
        outcome = result / num_test_col
    else:
        num_test_sty = np.array([211,5617,8564,125])
        outcome = result / num_test_sty
    return outcome

def select_images_for_hat(family,ylabel,index):
    list = []
    for each in family:
        if get_y_for_thisbatch([each],ylabel)[0] == index:
            list.append(each + 1)
    return list
        
#this part of code is to get the path of target file on pod
# def processDirectory (args, dirname, filenames):
#     print 'Directory',dirname
#     for filename in filenames:
#         print ' File',filename

In [ ]:
# os.path.walk(r'/datasets/ee285s-public/CelebA', processDirectory, None )
test_1 = plt.imread('/datasets/ee285s-public/CelebA/Img/img_align_celeba/202598.jpg')
test_2 = plt.imread('/datasets/ee285s-public/CelebA/Augmentation/202598.jpg')
plt.imshow(test_1)
plt.imshow(test_2)

In [ ]:
#because the dataset we used are not initially suitable for our project,so we do some preprocessing &
#selection for it,all the first-step processing are included in the function load_labels()
#index_to_use is the index of image qualified for future use.
ylabel_for_color,ylabel_for_style,index_to_use = load_labels()
#for the randomness,we do the shuffling to the index
np.random.shuffle(index_to_use)
#because the limitaion of the cpu,we can not load all the images and into matrix form,instead
#when we need them,we load the data.

##separate the data into train & validation & test sets
##after the first selection,one must lock the this code,to keep the purity of test data.
# train_family = index_to_use[0:(72557/10)*6]
# validation_family = index_to_use[(72557/10)*6:(72557/10)*8]
# test_family = index_to_use[(72557/10)*8:]
# torch.save(train_family,'train_family.pkl')
# torch.save(validation_family,'validation_family.pkl')
# torch.save(test_family,'test_family.pkl')

#load all the data from previous training
train_family = torch.load('train_family.pkl')
validation_family = torch.load('validation_family.pkl')
test_family = torch.load('test_family.pkl')

In [ ]:
#set all the hyperparameters(to be completed)
EPOCH = 20
LR = 0.01             
batch = 40 

net = Net_1()
net.cuda()
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LR)
# net.load_state_dict(torch.load('params.pkl'))

In [ ]:
net.load_state_dict(torch.load('params_col_ba50_1.pkl'))

In [ ]:
# variation_loss = torch.load('variation_loss.pkl')
# variation_loss_val = torch.load('variation_loss_val.pkl')

#training section
#load some datas & net parameters from previous training
running_loss = 0.0
running_loss_2 = 0.0
variation_loss = []
variation_loss_val = []
test_history_accuracy = []
test_history_accuracy_col = []
test_history_accuracy_sty = []
# variation_loss = torch.load('variation_loss_1.pkl')
# variation_loss_val = torch.load('variation_loss_val_1.pkl')
# test_history_accuracy = torch.load('test_history_accuracy.pkl')
# test_history_accuracy_col = torch.load('test_history_accuracy_col.pkl')
# test_history_accuracy_sty = torch.load('test_history_accuracy_sty.pkl')
# net.load_state_dict(torch.load('params.pkl'))
# print('paremeters loaded')

for epoch in range(EPOCH):
    net.eval()
    signal = False
    perbatch_index,num_batch = get_the_step_indexassignment(train_family,batch)
    #get the epoch accuracy,due to the limitation of gpu memory,we could only use part of the test data to do the computation     
    index_accur = validation_family
    np.random.shuffle(index_accur)
    n = len(index_accur)/15
    x = Variable(get_x_for_thisbatch(index_accur[0:n],False)).cuda()
    y_col = get_y_for_thisbatch(index_accur[0:n],ylabel_for_color).numpy()
#     y_sty = get_y_for_thisbatch(index_accur[0:n],ylabel_for_style).numpy()
    output_for_accuracy_col = net(x)
    prediction_col = onehot2label(output_for_accuracy_col.cpu().data.numpy().T)
#     prediction_sty = onehot2label(output_for_accuracy_sty.cpu().data.numpy().T)
    accuracy_of_this_epoch_1 = accuracy(prediction_col,y_col)
#     accuracy_of_this_epoch_2 = accuracy(prediction_sty,y_sty)
    test_history_accuracy_col.append(accuracy_of_this_epoch_1)
#     test_history_accuracy_sty.append(accuracy_of_this_epoch_2)
#     accuracy_of_this_epoch = accuracy_of_this_epoch_2 + accuracy_of_this_epoch_1
    print('accuracy of epoch for color',epoch+1,'=',accuracy_of_this_epoch_1)
#     print('accuracy of epoch for style',epoch+1,'=',accuracy_of_this_epoch_2)
#     torch.save(test_history_accuracy, 'test_history_accuracy.pkl')
    torch.save(test_history_accuracy_col,'test_history_accuracy_col_ba50.pkl')
    del x,y_col,output_for_accuracy_col
     
#     torch.save(test_history_accuracy_sty,'test_history_accuracy_sty.pkl')
    #early stopping to avoid overfitting
    if epoch > 0:
        if accuracy_of_this_epoch_1 < (test_history_accuracy_col[-1]+test_history_accuracy_col[-2])/2:
            print('early stopped')
            break;
    test_history_accuracy_col.append(accuracy_of_this_epoch_1)

    #train the model
    for step in range(num_batch*2):
        net.train()
        y_col = get_y_for_thisbatch(perbatch_index[step/2],ylabel_for_color)
#         y_sty = get_y_for_thisbatch(perbatch_index[step/2],ylabel_for_style)
        x = 0
        x = get_x_for_thisbatch(perbatch_index[step/2],signal)
        b_y_col = Variable(y_col).cuda()
#         b_y_sty = Variable(y_sty).cuda()
        b_x = Variable(x).cuda()
        output_col = net(b_x)             
        loss = loss_func(output_col, b_y_col)
        del b_x,b_y_col,output_col
        
        optimizer.zero_grad()           
        loss.backward()                 
        optimizer.step()
        
        #get the variation of lost of validation set
        #to save source,just compute partial batch size of it
        arr = validation_family             
        np.random.shuffle(arr)
        arr = arr[0:batch/2]
        net.eval()
        yval_temp_col = get_y_for_thisbatch(arr,ylabel_for_color)
#         yval_temp_sty = get_y_for_thisbatch(arr,ylabel_for_style)
        xval_temp = get_x_for_thisbatch(arr,False)
        b_y_col = Variable(yval_temp_col).cuda()
#         b_y_sty = Variable(yval_temp_sty).cuda()
        b_x_2 = Variable(xval_temp).cuda()
        output_col = net(b_x_2)             
        del b_x_2
        loss_2 = loss_func(output_col, b_y_col)
        del b_y_col,output_col
        
        running_loss += loss
        running_loss_2 += loss_2
        if step % 50 == 0:
            #to avoid recording some bad points.
            if running_loss.cpu().data[0] >= 5:
                variation_loss.append(running_loss.cpu().data.numpy())
            if running_loss_2.cpu().data[0] >= 5:
                variation_loss_val.append(running_loss_2.cpu().data.numpy())
                print(running_loss.cpu().data.numpy())                                      #loss per sample
            running_loss = 0.0
            running_loss_2 = 0.0
            torch.save(net.state_dict(), 'params_col_ba50_1.pkl')  
            torch.save(variation_loss, 'variation_loss_col_ba50_1.pkl')
            torch.save(variation_loss_val, 'variation_loss_val_col_ba50_1.pkl')
        #get some test prediction for comparsion 
        if step % 1000 == 0:
            pre_select = test_family
            np.random.shuffle(pre_select)
            xtest = get_x_for_thisbatch(pre_select[:10],False)
            x = Variable(xtest).cuda()
            y_col = get_y_for_thisbatch(pre_select[:10],ylabel_for_color).numpy()
#             y_sty = get_y_for_thisbatch(pre_select[:10],ylabel_for_style).numpy()
            output_for_accuracy_col = net(x)
            del x,xtest
            prediction_col = onehot2label(output_for_accuracy_col.cpu().data.numpy().T)
#             prediction_sty = onehot2label(output_for_accuracy_sty.cpu().data.numpy().T)

            print('the original label for color is',y_col)
            print('and the prediction for color is',prediction_col)
#             print('----------------------------------------------------')
#             print('the original label for style is',y_sty)
#             print('and the prediction for style is',prediction_sty)
    if signal == False:
        signal = True
    else:
        signal = False

In [ ]:
loss_col = torch.load('variation_loss_col_seperate_epoch8.pkl')
loss_val = torch.load('variation_loss_val_col_seperate_epoch80.pkl')
xx = range(len(loss_col))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
ll = torch.load('test_history_accuracy_col_ba50.pkl')